In [ ]:
"""
@author Michael Levet
@author Jeremy Alm
@date 08/18/2022

This code tests whether the relation algebra 33_65 
is representable on n points using a SAT solver. 
We define a Boolean formula \varphi whose satisfiability
is a necessary condition to be representable on n points.
So if \varphi is not satisfiable, then 34_65 is not representable
on n points.
"""

import boolexpr as bx
import itertools

import cProfile
import signal
from contextlib import contextmanager

In [ ]:
# change the value of n to test if 
# 34_65 is representable on n points
n = 24

vertices = list(range(n))
edges = list(itertools.combinations(vertices, 2))
ctx = bx.Context()
variables = { **{ (i,j,0) : ctx.get_var(str((i,j,0))) for i,j in edges} ,
             **{ (i,j,1) : ctx.get_var(str((i,j,1))) for i,j in edges} ,  
             **{ (i,j,2) : ctx.get_var(str((i,j,2))) for i,j in edges} }


In [ ]:
# exactly one color is true
conjuncts_colors = ( (((variables[(i,j,0)] | variables[(i,j,1)] | variables[(i,j,2)])) 
                    & ~(variables[(i,j,0)] & variables[(i,j,1)]) 
                    & ~(variables[(i,j,0)] & variables[(i,j,2)]) 
                    & ~(variables[(i,j,1)] & variables[(i,j,2)]))  
                    for i,j in edges )

phi_0 = bx.and_s(*conjuncts_colors)
print("Made phi_0")

R = {(0,1), (1,3), (0, 5), (1,8), (0,9), (0, 10), (1,10), (2, 4), (2, 6), (2, 7), (2, 8), (2, 9)} \
    | {(3, 6), (3, 7), (3, 8), (4,5), (4,6), (4, 7), (4, 8), (5, 7), (5, 9), (6, 7), (6, 9)}

B = {(0,2),(1,2),(0,3),(0,4),(1,5),(1,6)}
G = {(1,4), (0,6), (0,7), (1,7), (0, 8), (1,9)}

conjuncts_red_subgraph = (variables[(i,j,0)] for i,j in R)

conjuncts_blue_subgraph = (variables[(i,j,1)] for i,j in B)

conjuncts_green_subgraph = (variables[(i,j,2)] for i,j in G)


#color the existing edges by what we know
phi_1 = bx.and_s( bx.and_s(*conjuncts_red_subgraph), bx.and_s(*conjuncts_blue_subgraph),
                bx.and_s(*conjuncts_green_subgraph) )
print("Made phi_1")

def disjunction(i,j,c1,c2):
    # returns a disjunction that says edge ij has need c1-c2 met
    return bx.or_s( *( variables[(min(i,k), max(i,k), c1)] 
                               & variables[(min(j,k), max(j,k), c2)] for k in set(range(n)) - {i,j} ) ) 

# each edge must have its "needs" met

#def conjuncts_needs(c1, c2)

conjuncts_needs_red = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in itertools.product({0,1,2}, {0,1,2}) ))  
                       for i,j in R)


# all possible needs for blue edges
edge_needs_bl = {(0,0),(0,1),(0,2),(1,0),(2,0), (1, 1)}

# all possible needs for green edges
edge_needs_gr = {(0,0),(0,1),(0,2),(1,0),(2,0),(2,2)}

#every red edge must have its "needs" met
phi_2 = bx.and_s(*conjuncts_needs_red)

print("Made phi_2")
conjuncts_forbidden_triangles = (    ~(variables[(i,j,1)] & variables[(i,k,1)] & variables[(j,k,2)]) &
                                     ~(variables[(i,j,1)] & variables[(i,k,2)] & variables[(j,k,1)]) &
                                     ~(variables[(i,j,1)] & variables[(i,k,2)] & variables[(j,k,2)]) &
                                     ~(variables[(i,j,2)] & variables[(i,k,1)] & variables[(j,k,1)]) &
                                     ~(variables[(i,j,2)] & variables[(i,k,1)] & variables[(j,k,2)]) &
                                     ~(variables[(i,j,2)] & variables[(i,k,2)] & variables[(j,k,1)])
                                 for i,j,k in itertools.combinations(vertices,3))

#forbid all blue-green triangles except for ggg
phi_3 = bx.and_s(*conjuncts_forbidden_triangles)
print("Made phi_3")

#every edge must have its "needs" met
conjuncts_needs_Bl = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_bl ))  
                       for i,j in B)


#every blue edge must have its "needs" met
phi_4 = bx.and_s(*conjuncts_needs_Bl)
print("Made phi_4")

conjuncts_needs_Green = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_gr ))  
                       for i,j in G)


#every green edge must have its "needs" met
phi_5 = bx.and_s(*conjuncts_needs_Green)
print("Made phi_5")





# every edge not colored must have its "needs" met
edge_unknown = set(set(edges) - set(R) - set(B) - set(G))

def needs_r(i,j):
    # returns a disjunction that says edge ij is red and has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in itertools.product({0,1,2}, {0,1,2}) ))  

def needs_b(i,j):
    # returns a disjunction that says edge ij is blue has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_bl )) 

def needs_g(i,j):
    # returns a disjunction that says edge ij is blue has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_gr )) 


conjuncts_needs_unknown =  (  (variables[(i,j,0)] & needs_r(i,j) ) |  
                                (variables[(i,j,1)] & needs_b(i,j) ) |  
                                (variables[(i,j,2)] & needs_g(i,j) ) 
                              for i,j in edge_unknown)

phi_6 = bx.and_s(*conjuncts_needs_unknown)
print("Made phi_6 \n")


phi = bx.and_s(phi_0, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6)

print("Starting to simplify")
phi_simpl = phi.simplify()
print("Simplified")
phi_tseytin = phi.tseytin(ctx)
print("Tseytin Call Completed")
print("Call SAT")
phi_tseytin.sat()